In [1]:
import cv2
import os

def extract_frames_live(output_folder, frame_interval=1):
    # Open the webcam or external camera
    cap = cv2.VideoCapture(0)
    
    # Check if the camera is opened successfully
    if not cap.isOpened():
        print("Error: Unable to open camera.")
        return
    
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Frame index
    frame_index = 0
    
    # Loop for live video capturing and frame extraction
    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
        
        # Check if frame is retrieved successfully
        if not ret:
            print("Error: Unable to capture frame.")
            break
        
        # Save frame to output folder
        if frame_index % frame_interval == 0:
            frame_path = os.path.join(output_folder, f"frame_{frame_index}.jpg")
            cv2.imwrite(frame_path, frame)
        
        # Display the captured frame
        cv2.imshow('Live Frame', frame)
        
        # Check for key press (press 'q' to quit)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # Increment frame index
        frame_index += 1
    
    # Release the camera capture object
    cap.release()
    cv2.destroyAllWindows()

# Example usage
output_folder = "../Component 01/Dataset/Frames_Live"
frame_interval = 5

extract_frames_live(output_folder, frame_interval)

KeyboardInterrupt: 